In [ ]:
%matplotlib inline
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, nn, utils as gutils
import numpy as np
import pandas as pd
import d2lzh as d2l
import matplotlib.pyplot as plt
import time

In [ ]:
def _get_batch(batch, ctx):
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

In [ ]:
def train_CNN_class(train_iter, test_iter, net, loss, trainer, ctx, num_epochs,lr_period,lr_decay, end_con):
    print('training on', ctx)
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, m, start = 0.0, 0.0, 0, 0, time.time()
        if epoch > 0 and epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for i, batch in enumerate(train_iter):
            Xs, ys, batch_size = _get_batch(batch, ctx)
            ls = []
            with autograd.record():
                y_hats = [net(X) for X in Xs]
                ls = [loss(y_hat, y) for y_hat, y in zip(y_hats, ys)]
            for l in ls:
                l.backward()
            trainer.step(batch_size)
            train_l_sum += sum([l.sum().asscalar() for l in ls])
            n += sum([l.size for l in ls])       
        train_acc = evaluate_accuracy(train_iter, net, ctx)
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print('epoch %d, loss %.7f,cal %.4f,val %.4f, lr %.7f, time %.1f sec'
              % (epoch + 1, train_l_sum / n,train_acc, test_acc, trainer.learning_rate, time.time() - start))
        if test_acc>end_con:break

In [ ]:
cal = pd.read_csv('DT_train.csv',header=None)
val = pd.read_csv('DT_val.csv',header=None)
pre = pd.read_csv('DT_test.csv',header=None)

In [ ]:
cal_features = cal.iloc[:, 0:-1]
val_features = val.iloc[:, 0:-1]
pre_features = pre.iloc[:, 0:-1]
cal_labels = cal.iloc[:, -1]
val_labels = val.iloc[:, -1]
pre_labels = pre.iloc[:, -1]

In [ ]:
cal_features = nd.array(cal_features.values)
val_features = nd.array(val_features.values)
pre_features = nd.array(pre_features.values)

cal_labels = nd.array(cal_labels.values)
val_labels = nd.array(val_labels.values)
pre_labels = nd.array(pre_labels.values)

print(cal_features.shape,cal_labels.shape,val_features.shape,val_labels.shape,pre_features.shape,pre_labels.shape)

In [ ]:
batch_size=256

train_iter = gdata.DataLoader(gdata.ArrayDataset(cal_features, cal_labels), batch_size, shuffle=True)
val_iter  = gdata.DataLoader(gdata.ArrayDataset(val_features, val_labels), batch_size, shuffle=False)
pre_iter  = gdata.DataLoader(gdata.ArrayDataset(pre_features, pre_labels), batch_size, shuffle=False)

In [ ]:
class Attension(nn.Block):
    def __init__(self, c1, c2, **kwargs):
        super(Attension, self).__init__(**kwargs)        
        self.p1 = nn.Dense(c1, activation="tanh")
        self.p2 = nn.Dense(c2, activation="sigmoid")
    def forward(self, x):
        t1 = nd.softmax(self.p1(x))
        t2 = self.p2(t1)
        t3 = t2*x
        return nd.reshape(t3,(x.shape[0],1,x.shape[1]))

In [ ]:
loss = gloss.SoftmaxCrossEntropyLoss()
ctx = d2l.try_all_gpus()

net = nn.Sequential(prefix='WheatOr1')
net.add(
        Attension(32,200),
        nn.BatchNorm(axis=1, center=True, scale=True),
        nn.Conv1D(16, kernel_size=3, strides=1),
        nn.ELU(),
        nn.MaxPool1D(pool_size=2, strides=2),
        nn.BatchNorm(axis=1, center=True, scale=True),
        nn.Conv1D(16, kernel_size=3, strides=1),
        nn.ELU(),
        nn.BatchNorm(axis=1, center=True, scale=True),
        nn.Conv1D(16, kernel_size=3, strides=1),
        nn.ELU(),
        nn.Dense(512, activation="relu"),nn.BatchNorm(axis=1, center=True, scale=True),
        nn.Dense(128, activation="relu"), nn.BatchNorm(axis=1, center=True, scale=True),
        nn.Dense(30))
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

In [ ]:
net.collect_params().reset_ctx(ctx)
net.hybridize()

In [ ]:
train_epochs=500
train_lr=0.0005
lr_period=100
lr_decay=0.1
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': train_lr})

In [ ]:
train_CNN_class(train_iter, val_iter, net, loss, trainer, ctx, train_epochs,lr_period,lr_decay, 0.93)

In [ ]:
train_acc = evaluate_accuracy(train_iter, net, ctx)
val_acc = evaluate_accuracy(val_iter, net, ctx)
test_acc = evaluate_accuracy(pre_iter, net, ctx)
print('train_acc','val_acc','test_acc')
print(train_acc,val_acc,test_acc)